## Proy 2


Importación y transformación de los datos

In [104]:
import pandas as pd

columnas = ["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num"]
datos_iniciales = pd.read_csv('https://raw.githubusercontent.com/karen0c/Proyecto_1_ACTD/main/processed.cleveland.data',header=None, names=columnas, na_values="?")

datos_iniciales = datos_iniciales.dropna().reset_index(drop=True) #elimina filas con valores faltantes
#print(datos_iniciales.head())
#print(datos_iniciales.tail())

# crear unos nuevos datos donde guardaremos la información con los datos discretizados
datos = datos_iniciales

for i in range(0,297):
  if datos_iniciales.loc[i, 'age'] <= 40:
    datos.loc[i, 'age'] = 1
  elif datos_iniciales.loc[i, 'age'] > 40 and datos_iniciales.loc[i, 'age'] <= 50:
    datos.loc[i, 'age'] = 2
  elif datos_iniciales.loc[i, 'age'] > 50 and datos_iniciales.loc[i, 'age'] <= 60:
    datos.loc[i, 'age'] = 3
  else:
    datos.loc[i, 'age'] = 4

for i in range(0,297):
  if datos_iniciales.loc[i, "trestbps"] <= 120:
    datos.loc[i, "trestbps"] = 1
  elif datos_iniciales.loc[i, "trestbps"] > 120 and datos_iniciales.loc[i, "trestbps"] <= 139:
    datos.loc[i, "trestbps"] = 2
  elif datos_iniciales.loc[i, "trestbps"] >= 140 and datos_iniciales.loc[i, "trestbps"] <= 159:
    datos.loc[i, "trestbps"] = 3
  elif datos_iniciales.loc[i, "trestbps"] >= 160 and datos_iniciales.loc[i, "trestbps"] <= 179:
    datos.loc[i, "trestbps"] = 4
  else:
    datos.loc[i, "trestbps"] = 5
    
for i in range(0,297):
  if datos_iniciales.loc[i, "chol"] <= 200:
    datos.loc[i, "chol"] = 1
  elif datos_iniciales.loc[i, "chol"] > 200 and datos_iniciales.loc[i, "chol"] < 240:
    datos.loc[i, "chol"] = 2
  else:
    datos.loc[i, "chol"] = 3
    
for i in range(0,297):
  if datos_iniciales.loc[i, "thalach"] <= 120:
    datos.loc[i, "thalach"] = 1
  elif datos_iniciales.loc[i, "thalach"] > 120 and datos_iniciales.loc[i, "thalach"] <= 140:
    datos.loc[i, "thalach"] = 2
  elif datos_iniciales.loc[i, "thalach"] > 140 and datos_iniciales.loc[i, "thalach"] < 160:
    datos.loc[i, "thalach"] = 3
  else:
    datos.loc[i, "thalach"] = 4
    
for i in range(0,297):
  if datos_iniciales.loc[i, "oldpeak"] <= 1:
    datos.loc[i, "oldpeak"] = 1
  elif datos_iniciales.loc[i, "oldpeak"] > 1 and datos_iniciales.loc[i, "oldpeak"] <= 2:
    datos.loc[i, "oldpeak"] = 2
  else:
    datos.loc[i, "oldpeak"] = 3
    
for i in datos_iniciales.index:
  if datos_iniciales.loc[i, "num"] == 0:
    datos.loc[i, "num"] = 0
  else:
    datos.loc[i, "num"] = 1

#USAR SOLO EL 80% DE LOS DATOS, PARA QUE SIRVAN COMO ENTRENAMIENTO, LO DEMÁS SE USARÁ COMO TEST
sep = int(0.8*len(datos))
datos_train = datos[:sep]
#print(datos_train.head(10))
#print(datos_train.tail())

In [62]:
print(sep)

237


In [105]:
datos_prueba = datos[sep:]
#print(datos_prueba.head(10))

Estime la estructura del modelo usando el método por restricciones

In [106]:
from pgmpy.estimators import PC
est = PC(data=datos_train)

estimated_model = est.estimate(variant="stable", max_cond_vars=3)
print(estimated_model)
print(estimated_model.nodes())
print(estimated_model.edges())

  0%|          | 0/3 [00:00<?, ?it/s]


['thal', 'num', 'exang', 'cp', 'oldpeak', 'slope', 'ca', 'age', 'trestbps', 'restecg', 'sex']
[('thal', 'num'), ('exang', 'cp'), ('slope', 'oldpeak'), ('age', 'ca'), ('restecg', 'trestbps'), ('sex', 'thal')]


Se convierte el objeto DAG obtenido con el anterior procedimiento a una red bayesiana y se usa el estimador de máxima verosimilitud para estimar los parámetros de la red.

In [107]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

estimated_model = BayesianNetwork(estimated_model)
estimated_model.fit(data=datos_prueba, estimator = MaximumLikelihoodEstimator) 
#for i in estimated_model.nodes():
 #   print(estimated_model.get_cpds(i))

### Predicción

In [108]:
# Importar librerías necesarias
from pgmpy.inference import VariableElimination

In [109]:
# Crear una lista vacía para contener las variables del modelo
variables_modelo = []

# Recorrer cada variable del modelo y agregarla a la lista si está presente en la base de datos original
for nodo in estimated_model.nodes():
    if nodo in datos_prueba.columns:
        variables_modelo.append(datos_prueba[nodo])

# Concatenar todas las variables en una sola base de datos
nueva_base = pd.concat(variables_modelo, axis=1)
#print(nueva_base.tail())
#print(len(nueva_base))


In [110]:
# conversión de respuesta de datos de prueba a una variable binaria
# 0-> no hay enfermedad; 1-> hay presencia de enfermedad
bin_test = []

for i in nueva_base.index:
  if nueva_base.loc[i, "num"] == 0:
    bin_test.append(0)      
  else:
    bin_test.append(1)      

In [111]:
print(bin_test)

[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


In [112]:
# Crear un objeto de inferencia basado en el modelo entrenado
infer = VariableElimination(estimated_model)
bin_modelo = []
# Calcular las predicciones para cada fila en los datos de prueba
for i in (nueva_base.index):
    evidence = nueva_base.loc[i].to_dict()
    evidence.pop('num', None)  # Elimina la clave 'num' del diccionario si está presente
    prediccion = infer.query(['num'], evidence=evidence)
    #print(f'Predicción para la fila {i}: {prediccion.values}')

    
    posicion = -1
    probabilidad = -1
    for j in range(0,2):
        if prediccion.values[estimated_model.get_cpds("num").state_names["num"].index(j)]>probabilidad:
            probabilidad = prediccion.values[estimated_model.get_cpds("num").state_names["num"].index(j)]
            posicion = j
            
    if posicion == 0:
        bin_modelo.append(0)
    else:
        bin_modelo.append(1)


In [77]:
print(bin_modelo)

[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0]


In [113]:
verd_positivo = 0
verd_negativo = 0
falso_positivo = 0
falso_negativo = 0

#contar el número de verdaderos positivos y negativos            
for i in range(len(nueva_base)):
    if bin_test[i]==bin_modelo[i]:
        if bin_modelo[i]==0:
            verd_positivo = verd_positivo+1
        else:
            verd_negativo = verd_negativo+1
    else:
        if bin_modelo[i]==0:
            falso_positivo = falso_positivo+1
        else:
            falso_negativo = falso_negativo+1


In [114]:
print(verd_positivo, verd_negativo, falso_positivo, falso_negativo)

24 22 9 5


## Estime la estructura del modelo usando el método por puntajes

In [115]:
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score

scoring_method = K2Score(data=datos_prueba)
esth = HillClimbSearch(data=datos_prueba)
estimated_modelK2 = esth.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
)
print(estimated_modelK2)
print(estimated_modelK2.nodes())
print(estimated_modelK2.edges())

  0%|          | 0/10000 [00:00<?, ?it/s]


['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[('age', 'chol'), ('sex', 'chol'), ('fbs', 'thalach'), ('fbs', 'ca'), ('fbs', 'thal'), ('restecg', 'trestbps'), ('restecg', 'thal'), ('restecg', 'age'), ('thalach', 'chol'), ('exang', 'cp'), ('exang', 'thalach'), ('exang', 'thal'), ('exang', 'oldpeak'), ('oldpeak', 'thalach'), ('oldpeak', 'slope'), ('slope', 'chol'), ('ca', 'num'), ('thal', 'oldpeak'), ('thal', 'trestbps'), ('thal', 'sex'), ('num', 'thal'), ('num', 'exang')]


In [116]:
print(scoring_method.score(estimated_modelK2))

-760.8422067863243


In [117]:
estimated_modelK2 = BayesianNetwork(estimated_modelK2)
estimated_modelK2.fit(data=datos_prueba, estimator = MaximumLikelihoodEstimator) 

print(estimated_modelK2)
print(estimated_modelK2.nodes())
print(estimated_modelK2.edges())

#for i in estimated_modelK2.nodes():
    #print(estimated_modelK2.get_cpds(i))


['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[('age', 'chol'), ('sex', 'chol'), ('fbs', 'thalach'), ('fbs', 'ca'), ('fbs', 'thal'), ('restecg', 'trestbps'), ('restecg', 'thal'), ('restecg', 'age'), ('thalach', 'chol'), ('exang', 'cp'), ('exang', 'thalach'), ('exang', 'thal'), ('exang', 'oldpeak'), ('oldpeak', 'thalach'), ('oldpeak', 'slope'), ('slope', 'chol'), ('ca', 'num'), ('thal', 'oldpeak'), ('thal', 'trestbps'), ('thal', 'sex'), ('num', 'thal'), ('num', 'exang')]


Se imprime el resultado del puntaje obtenido

In [118]:
print(scoring_method.score(estimated_modelK2))

-760.8422067863244


In [119]:
# Crear una lista vacía para contener las variables del modelo
variables_modeloK2 = []

# Recorrer cada variable del modelo y agregarla a la lista si está presente en la base de datos original
for nodo in estimated_modelK2.nodes():
    if nodo in datos_prueba.columns:
        variables_modeloK2.append(datos_prueba[nodo])

# Concatenar todas las variables en una sola base de datos
nueva_baseK2 = pd.concat(variables_modeloK2, axis=1)
print(nueva_baseK2.head())

     age  sex   cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
237  2.0  1.0  2.0       1.0   2.0  0.0      0.0      3.0    0.0      1.0   
238  2.0  0.0  2.0       2.0   3.0  0.0      0.0      4.0    0.0      1.0   
239  2.0  0.0  4.0       2.0   3.0  0.0      0.0      4.0    0.0      1.0   
240  4.0  1.0  1.0       2.0   2.0  0.0      0.0      3.0    0.0      3.0   
241  3.0  0.0  3.0       1.0   1.0  1.0      0.0      1.0    0.0      1.0   

     slope   ca  thal  num  
237    1.0  0.0   3.0    0  
238    1.0  0.0   3.0    0  
239    1.0  0.0   3.0    0  
240    2.0  2.0   3.0    1  
241    1.0  0.0   3.0    0  


In [121]:
# Crear un objeto de inferencia basado en el modelo entrenado
inferK2 = VariableElimination(estimated_modelK2)
bin_modeloK2 = []

# Calcular las predicciones para cada fila en los datos de prueba
for i in (nueva_baseK2.index):
    evidenceK2 = nueva_baseK2.loc[i].to_dict()
    evidenceK2.pop('num', None)  # Elimina la clave 'num' del diccionario si está presente
    prediccionK2 = inferK2.query(['num'], evidence=evidenceK2)
    #print(f'Predicción para la fila {i}: {prediccionK2.values}')

    
    posicion = -1
    probabilidad = -1
    for j in range(0,2):
        if prediccionK2.values[estimated_modelK2.get_cpds("num").state_names["num"].index(j)]>probabilidad:
            probabilidad = prediccionK2.values[estimated_modelK2.get_cpds("num").state_names["num"].index(j)]
            posicion = j
    
    if posicion == 0:
        bin_modeloK2.append(0)
    else:
        bin_modeloK2.append(1)


In [122]:
print(bin_modeloK2)

[0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


In [123]:
verd_positivo = 0
verd_negativo = 0
falso_positivo = 0
falso_negativo = 0

#contar el número de verdaderos positivos y negativos            
for i in range(len(nueva_baseK2)):
    if bin_test[i]==bin_modeloK2[i]:
        if bin_modeloK2[i]==0:
            verd_positivo = verd_positivo+1
        else:
            verd_negativo = verd_negativo+1
    else:
        if bin_modeloK2[i]==0:
            falso_positivo = falso_positivo+1
        else:
            falso_negativo = falso_negativo+1


In [124]:
print(verd_positivo, verd_negativo, falso_positivo, falso_negativo)

25 29 2 4


### El procedimiento anterior usando el punta BIC

In [125]:
from pgmpy.estimators import BicScore

scoring_method = BicScore(data=datos_prueba)
esth = HillClimbSearch(data=datos_prueba)
estimated_modelBic = esth.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)

)
print(estimated_modelBic)
print(estimated_modelBic.nodes())
print(estimated_modelBic.edges())

  0%|          | 0/10000 [00:00<?, ?it/s]


['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[('exang', 'cp'), ('exang', 'oldpeak'), ('exang', 'slope'), ('oldpeak', 'thalach'), ('thal', 'num'), ('thal', 'sex'), ('num', 'ca'), ('num', 'exang')]


Se imprime el resultado del puntaje obtenido

In [126]:
print(scoring_method.score(estimated_modelBic))

-805.8697998114878


In [127]:
estimated_modelBic = BayesianNetwork(estimated_modelBic)
estimated_modelBic.fit(data=datos_prueba, estimator = MaximumLikelihoodEstimator) 

print(estimated_modelBic)
print(estimated_modelBic.nodes())
print(estimated_modelBic.edges())

#for i in estimated_modelK2.nodes():
    #print(estimated_modelK2.get_cpds(i))


['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[('exang', 'cp'), ('exang', 'oldpeak'), ('exang', 'slope'), ('oldpeak', 'thalach'), ('thal', 'num'), ('thal', 'sex'), ('num', 'ca'), ('num', 'exang')]


In [128]:
print(scoring_method.score(estimated_modelBic))

-805.8697998114878


In [129]:
# Crear una lista vacía para contener las variables del modelo
variables_modeloBic = []

# Recorrer cada variable del modelo y agregarla a la lista si está presente en la base de datos original
for nodo in estimated_modelBic.nodes():
    if nodo in datos_prueba.columns:
        variables_modeloBic.append(datos_prueba[nodo])

# Concatenar todas las variables en una sola base de datos
nueva_baseBic = pd.concat(variables_modeloBic, axis=1)
print(nueva_baseBic.head())

     age  sex   cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
237  2.0  1.0  2.0       1.0   2.0  0.0      0.0      3.0    0.0      1.0   
238  2.0  0.0  2.0       2.0   3.0  0.0      0.0      4.0    0.0      1.0   
239  2.0  0.0  4.0       2.0   3.0  0.0      0.0      4.0    0.0      1.0   
240  4.0  1.0  1.0       2.0   2.0  0.0      0.0      3.0    0.0      3.0   
241  3.0  0.0  3.0       1.0   1.0  1.0      0.0      1.0    0.0      1.0   

     slope   ca  thal  num  
237    1.0  0.0   3.0    0  
238    1.0  0.0   3.0    0  
239    1.0  0.0   3.0    0  
240    2.0  2.0   3.0    1  
241    1.0  0.0   3.0    0  


In [130]:

# Crear un objeto de inferencia basado en el modelo entrenado
inferBic = VariableElimination(estimated_modelBic)
bin_modeloBic = []

# Calcular las predicciones para cada fila en los datos de prueba
for i in (nueva_baseBic.index):
    evidenceBic = nueva_baseBic.loc[i].to_dict()
    #print(evidenceBic)
    evidenceBic.pop('num', None)  # Elimina la clave 'num' del diccionario si está presente
    prediccionBic = inferBic.query(['num'], evidence=evidenceBic)
    #print(f'Predicción para la fila {i}: {prediccionBic.values}')

    
    posicion = -1
    probabilidad = -1
    for j in range(0,2):
        if prediccionBic.values[estimated_modelBic.get_cpds("num").state_names["num"].index(j)]>probabilidad:
            probabilidad = prediccionBic.values[estimated_modelBic.get_cpds("num").state_names["num"].index(j)]
            posicion = j
    
    if posicion == 0:
        bin_modeloBic.append(0)
    else:
        bin_modeloBic.append(1)

In [131]:
print(bin_modeloBic)

[0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0]


In [132]:
verd_positivo = 0
verd_negativo = 0
falso_positivo = 0
falso_negativo = 0

#contar el número de verdaderos positivos y negativos            
for i in range(len(nueva_baseBic)):
    if bin_test[i]==bin_modeloBic[i]:
        if bin_modeloBic[i]==0:
            verd_positivo = verd_positivo+1
        else:
            verd_negativo = verd_negativo+1
    else:
        if bin_modeloBic[i]==0:
            falso_positivo = falso_positivo+1
        else:
            falso_negativo = falso_negativo+1


In [133]:
print(verd_positivo, verd_negativo, falso_positivo, falso_negativo)

24 27 4 5


## El mejor modelo por método de aprendizaje

In [136]:

bic_score2 = BicScore(data=datos_train).score(estimated_modelK2)
print(bic_score2)

-3719.4516965632365
